In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn import ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import time


# подключу гугл диск
from google.colab import drive
drive.mount('/content/drive')


# тренировочный датасет
df_train = pd.read_csv('drive/MyDrive/Colab Notebooks/Healthcare/train_data__df2catencode_ultimate.csv', index_col=0)

#df_train['Stay'] = df_train['Stay'].map({
#    5: 'very_fast', 15: 'fast', 25:'normal', 35:'long_normal', 45:'very_long_normal',
#    55:'long', 65:'very_long', 75:'super_very_long', 85:'small_infinity',95:'infinity',
#    105:'big_infinity'
#    })

# Разделю датасет на X  и Y - входные и выходные данные
#X = pd.read_csv('drive/MyDrive/Colab Notebooks/Healthcare/train_data__clean_catboost__drop_addiction__X.csv', index_col=0)
#Y = pd.read_csv('drive/MyDrive/Colab Notebooks/Healthcare/train_data_clean4cbe__Y.csv', index_col=0)
X = df_train.drop("Stay", axis=1)
Y = df_train.Stay

# Split the dataset: 80% vs 20%
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=123)

Mounted at /content/drive


In [ ]:
X_train

,Hospital_code,Bed_Grade,Severity_of_Illness,Visitors_with_Patient,Age,Department,Ward_Type,Type_of_Admission,Ward_Facility_Code_At_Hospital_type_code
272063,30,1,3,2,25,32.687744,31.961994,30.866845,50.371842
276353,18,3,1,2,5,31.342491,28.379625,34.209386,50.313864
281916,25,3,1,3,15,32.615316,31.902956,30.636403,46.371515
11242,28,2,2,4,25,35.037205,34.252311,33.596862,43.306356
158646,24,3,2,2,30,33.228461,32.398449,31.327630,51.513706
...,...,...,...,...,...,...,...,...,...
192476,28,3,1,3,25,33.126530,32.287625,34.435190,45.663096
17730,26,3,2,5,45,34.525217,33.818615,35.467154,52.435781
28030,28,2,2,3,45,33.803160,32.896124,34.894277,45.031545
277869,21,4,2,2,75,32.645667,31.923854,34.201263,50.338497


In [ ]:
Y_train

272063    25
276353    25
281916    15
11242     55
158646    25
          ..
192476    15
17730     75
28030     25
277869    25
249342     5
Name: Stay, Length: 254750, dtype: int64

In [ ]:
params = {'n_estimators': 500,
          'max_depth': 4,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'squared_error'}

In [ ]:
reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(X_train, Y_train)

GradientBoostingRegressor(learning_rate=0.01, max_depth=4, min_samples_split=5,
                          n_estimators=500)

In [ ]:
mse = mean_squared_error(Y_test, reg.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

The mean squared error (MSE) on test set: 252.3709


In [ ]:
rmse = np.sqrt(mse)
print("The mean squared error (RMSE) on test set: {:.4f}".format(rmse))

The mean squared error (RMSE) on test set: 15.8862


> Резюме: среднеквадраичная ошибка (15.9) при 500 деревьях. Это очень и очень существенная величина. Модель ошибается на два - три диапазона, то есть, например, модель предскажет, что пациент будет лежать 45 дней, а он может пролежать от 30 до 60 дней (это три класса в нашей модели)